In [1]:
import sys
import plotly.express as px
import numpy as np
from gymnasium.wrappers import TimeLimit

sys.path.append('..')

from environment.mangoEnv import MangoEnv, MAX_AMOUNT

In [2]:
render_mode = None
ext_mngo_price=0.1
init_mngo_pool_balance=1e4 / MAX_AMOUNT
init_usdc_pool_balance=1e3 / MAX_AMOUNT
init_treasury_size_usdc=120e6 / MAX_AMOUNT
mngo_collateral_factor=1.5
arb_efficiency_factor=0.5

env = MangoEnv(
    render_mode=render_mode, 
    ext_mngo_price_mean=ext_mngo_price, 
    init_mngo_pool_balance=init_mngo_pool_balance,
    init_usdc_pool_balance=init_usdc_pool_balance,
    init_treasury_size_usdc=init_treasury_size_usdc,
    mngo_collateral_factor=mngo_collateral_factor,
    arb_efficiency_factor=arb_efficiency_factor    
)

env = TimeLimit(env, max_episode_steps=100)

In [3]:
env.reset()
total_reward = 0
done = False
step = 0

while not done:
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    step += 1
    done = terminated or truncated
print(f"Steps passed: {step}, total reward: {total_reward}")
print(f"Health_factor: {env.get_wrapper_attr('mango').get_user_health_factor()}")

Steps passed: 100, total reward: 2.0
Health_factor: 0.0


In [4]:
env.reset()
total_reward = 0
done = False

smart_actions = (
    (5, 18e6 / MAX_AMOUNT), # Buy MNGO on external market
    (1, 18e6 / MAX_AMOUNT), # Put MNGO as collateral
    (0, 1.4e4 / MAX_AMOUNT), # Swap USDC to MNGO
    (2, 120e6 / MAX_AMOUNT), # Borrow
    (4, 0.), # Just Relax
    (4, 0.),
    (4, 0.),
    (4, 0.),
    (4, 0.),
    (4, 0.)
)

for ii, action in enumerate(smart_actions):
    observation, reward, terminated, truncated, info = env.step(action)
    print(f"reward at {ii} step: {reward}")
    total_reward += reward
    done = terminated or truncated
print(f"Done: {done}, total reward: {total_reward}")
print(f"Treasury: {env.get_wrapper_attr('mango').treasury_usdc}")
print(f"Health_factor: {env.get_wrapper_attr('mango').get_user_health_factor()}")

reward at 0 step: 0
reward at 1 step: 0
reward at 2 step: 0
reward at 3 step: 10.0
reward at 4 step: 0
reward at 5 step: 0
reward at 6 step: 0
reward at 7 step: 0
reward at 8 step: 0
reward at 9 step: 0
Done: False, total reward: 10.0
Treasury: 0.12209551456987268
Health_factor: 0.0
